In [25]:
import torch
import torch.nn as nn
# from torch.nn import init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as tv_utils
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision
import torch.autograd as autograd
from PIL import Image
import imp
import os
import sys
import math
import time
import random
import shutil
import cv2
import scipy.misc
from glob import glob
import sklearn
import logging

from tqdm import tqdm
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')

%matplotlib inline

/home/shubham/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [48]:
if not torch.cuda.is_available():
    print("SORRY: No CUDA device")

SORRY: No CUDA device


In [43]:
imageSize = 64
batchSize = 64

In [50]:
nz = 100
ngf = 64
ndf = 64
nc = 3

In [49]:
transform = transforms.Compose([transforms.Scale(imageSize),
                    transforms.CenterCrop(imageSize),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])

dataset = dset.MNIST('MNIST', transform=transform)

dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True)

In [83]:
nn.ConvTranspose2d?

In [93]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(
            
            # input: 100x1x1
            nn.ConvTranspose2d(nz, 512, 4, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            # state: 512x4x4
            nn.ConvTranspose2d(512, 256, 4, 2),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            # state: 256x8x8
            nn.ConvTranspose2d(256, 128, 4, 2),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            # state: 128x16x16
            nn.ConvTranspose2d(128, nc, 4, 2),
            nn.Tanh(),
            
            # state: ncx32x32    
        )
    
    def forward(self, input):
        return self.main(input)

netG = _netG()
netG

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2))
    (10): Tanh ()
  )
)

In [105]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            
            # input: ncx64x64
            nn.Conv2d(nc, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state: 64x32x32
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            
            # state: 128x16x16
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            # state: 256x8x8
            nn.Conv2d(256, 1, 8, 1, 0),
            nn.Sigmoid(),
            
            # state: 1x1x1
        )
    
    def forward(self, input):
        return self.main(input)

netD = _netD()
netD

_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): ReLU (inplace)
    (8): Conv2d(256, 1, kernel_size=(8, 8), stride=(1, 1))
    (9): Sigmoid ()
  )
)